In [ ]:
from carbonai import PowerMeter
power_meter = PowerMeter(project_name="Hackhaton - Training script", filepath="/home/jovyan/tmp/power_consumption.csv")

In [ ]:
@power_meter.measure_power(
   package="sklearn",
   algorithm="RandomForestClassifier",
   data_type="tabular",
   algorithm_params="n_estimators=300, max_depth=15",
   comments="Classifier trained on the MNIST dataset, 3rd test"
)
def my_func():
   print("toto")

In [ ]:
power_meter

In [ ]:
from libs import preprocessing as pp
from libs.utils import imageToTensor, crop_imgs, draw_img_boxes, box_dataset_generator
from carDetector import carDetector
from carClassifier import carClassifier

In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
home = "/home/jovyan/activities_data/hi__paris_2022_hackathon/final_challenge/datasets"
footprint_path = f"{home}/car_models_footprint.csv"
car_path = f"{home}/datasets_train/train_annotation/_annotation.csv"
imgs_path = f"{home}/datasets_train/train/"
tmp_folder = f"/home/jovyan/tmp"

In [ ]:
with power_meter(
  package="sklearn",
  algorithm="RandomForestClassifier",
  data_type="tabular",
  data_shape=<your_data>.shape,
  algorithm_params="n_estimators=300, max_depth=15",
  comments="Classifier trained on the MNIST dataset, 3rd test"
):

# Parameters

In [ ]:
BATCH_SIZE = 1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading datasets

In [ ]:
x, y = pp.create_car_dataset(footprint_path, car_path, label="brand")
img_dataset = pp.ImageDataset(imgs_path, footprint_path, car_path, num_pixels=512, min_max_scaling=True)

n_images = len(img_dataset)
print(f"Containing {n_images} images")

# Train / Test split

In [ ]:
train_proportion = 0.7

num_train = int(len(img_dataset) * train_proportion)
num_test = len(img_dataset) - num_train

# Datasets
train_set, val_set = torch.utils.data.random_split(img_dataset, [num_train, num_test])

# Dataloaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE)

# Loading models

In [ ]:
car_detector = carDetector(min_prob=4e-1, car_idx=3)
car_classifier = carClassifier()

# Loading Box dataset

In [ ]:
generate_images = False:
    
if generate_images:
    X_train, y_train = box_dataset_generator(
        train_set,
        f"{tmp_folder}/train",
        car_detector,
        DEVICE
    )

    X_test, y_test = box_dataset_generator(
        val_set,
        f"{tmp_folder}/test",
        car_detector,
        DEVICE
    )

    train_df = pd.DataFrame({"X":X_train, "y":y_train}, columns=["X", "y"]).to_csv(f"{tmp_folder}/train/record.csv")
    test_df = pd.DataFrame({"X":X_test, "y":y_test}, columns=["X", "y"]).to_csv(f"{tmp_folder}/test/record.csv")
else:
    train_df = pd.read_csv(f"{tmp_folder}/train/record.csv")
    X_train, y_train = train_df["X"].tolist(), train_df["y"].tolist()
    test_df = pd.read_csv(f"{tmp_folder}/test/record.csv")
    X_test, y_test = test_df["X"].tolist(), test_df["y"].tolist()
    
train_boxset = pp.boxDataset2(X_train, y_train)
val_boxset = pp.boxDataset2(X_test, y_test)

train_boxloader = DataLoader(train_boxset, shuffle=True, batch_size=16)
val_boxloader = DataLoader(val_boxset, shuffle=True, batch_size=16)

# Training algorithm

In [ ]:
from tqdm import tqdm

n_epochs = 300
losses = []
pbar = tqdm(range(1, n_epochs+1))

car_classifier.to(DEVICE)

for epoch in pbar:
    epoch_loss = 0
    for x, y in train_boxloader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        loss = car_classifier.fit(x, y)

        pbar.set_postfix({
            'batch loss': '{0:.2f}'.format(loss)
        })

        epoch_loss += loss

    epoch_loss /= len(train_boxloader)
    pbar.set_description('{:.5e}'.format(epoch_loss))

    losses.append(epoch_loss)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(losses)

In [ ]:
import os
torch.save(car_classifier.state_dict(), os.path.abspath('./my_work/team-03-084-submission-ab/Code/models/car_classifier_1.pth'))

# Evaluation

In [ ]:
import torch

y_true = []
y_pred = []

car_classifier = car_classifier.to("cpu")
for x, y in val_boxloader:
    y_ = car_classifier.predict(x)
    y_true.append(y)
    y_pred.append(torch.argmax(y_, axis=1))
    
y_true = torch.concat(y_true)
y_pred = torch.concat(y_pred)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay